In [2]:
import os

################################################################
# Principi di Shannon e la Struttura:

# Confusione: Oscurare la relazione tra chiave e testo cifrato (tramite S-Box).
# Diffusione: Distribuire l'influenza di un bit del testo in chiaro su molti bit del testo cifrato (tramite P-Box o permutazioni).

# Cifrario a Blocchi Iterativo: Opereremo su blocchi di dati di dimensione fissa attraverso multipli "round".
# Key Schedule: Genereremo sotto-chiavi di round dalla chiave principale.
# Componenti del Round (come prima):

# Key Addition: XOR del blocco con la chiave di round.
# Substitution: Sostituzione byte-per-byte usando una S-Box.
# Permutation: Mescolamento dei byte nel blocco.

################################################################

# --- DEFINIZIONI PRELIMINARI ---
BLOCK_SIZE_BYTES = 16  # 128 bit
KEY_SIZE_BYTES = 16    # 128 bit
NUM_ROUNDS = 10        # AES-128

# --- NON USARE IN PRODUZIONE, CODICE A SCOPO DI STUDIO ---

# Esempio di S-Box (DEVE ESSERE PROGETTATA CORRETTAMENTE!)
# QUESTA È SOLO UN PLACEHOLDER E NON È SICURA
S_BOX = list((i * 17 + 31) % 256 for i in range(256)) # Esempio molto ingenuo

# Genera l'inversa della S-Box
INV_S_BOX = [0] * 256
for i, val in enumerate(S_BOX):
    INV_S_BOX[val] = i

def key_schedule(master_key: bytes) -> list[bytes]:
    """
    Espande la chiave principale in chiavi di round.
    ATTENZIONE: Questa è una key schedule ESTREMAMENTE INSICURA, solo per esempio.
    """
    if len(master_key) != KEY_SIZE_BYTES:
        raise ValueError(f"La chiave deve essere di {KEY_SIZE_BYTES} byte")

    round_keys = []
    current_key = bytearray(master_key)

    for i in range(NUM_ROUNDS + 1): # +1 per la chiave iniziale o pre-whitening
        # Esempio molto ingenuo: ruota e XORa con una costante di round
        # NON USARE IN PRODUZIONE!
        round_keys.append(bytes(current_key))
        # Semplice modifica per la prossima chiave di round (NON CRITTOGRAFICAMENTE SICURA)
        current_key.append(current_key.pop(0)) # Rotazione
        for j in range(len(current_key)):
            current_key[j] ^= (i+1) # XOR con una costante di round fittizia
    return round_keys

def add_round_key(state: bytearray, round_key: bytes):
    """Combina lo stato con la chiave di round tramite XOR."""
    for i in range(len(state)):
        state[i] ^= round_key[i % len(round_key)] # Gestisce chiavi di round di lunghezza diversa (non ideale)

def substitute_bytes(state: bytearray, s_box: list[int]):
    """Applica la S-Box a ogni byte dello stato."""
    for i in range(len(state)):
        state[i] = s_box[state[i]]

def permute_bytes(state: bytearray):
    """
    Permuta i byte dello stato per diffusione.
    Esempio molto semplice: una rotazione del blocco.
    IN PRATICA, DEVE ESSERE UNA PERMUTAZIONE PIÙ ROBUSTA.
    """
    # Esempio: shift ciclico semplice (ShiftRows semplificato)
    # Per un blocco 4x4 (16 byte):
    # Riga 0: nessuna modifica
    # Riga 1: shift di 1 a sinistra
    # Riga 2: shift di 2 a sinistra
    # Riga 3: shift di 3 a sinistra
    # Questo è solo un esempio, la logica di AES è più complessa
    # e opera su colonne dopo ShiftRows. Qui semplifichiamo.

    if len(state) == 16: # Assumendo un blocco 4x4 per questa permutazione specifica
        # Riga 1 (indici 4-7)
        temp_row1 = state[4:8]
        state[4:8] = temp_row1[1:] + temp_row1[:1]
        # Riga 2 (indici 8-11)
        temp_row2 = state[8:12]
        state[8:12] = temp_row2[2:] + temp_row2[:2]
        # Riga 3 (indici 12-15)
        temp_row3 = state[12:16]
        state[12:16] = temp_row3[3:] + temp_row3[:3]
    else:
        # Permutazione generica più semplice per altre dimensioni: rotazione dell'intero blocco
        if len(state) > 0:
            first_byte = state.pop(0)
            state.append(first_byte)


def inverse_substitute_bytes(state: bytearray, inv_s_box: list[int]):
    """Applica l'inversa della S-Box a ogni byte dello stato."""
    for i in range(len(state)):
        state[i] = inv_s_box[state[i]]

def inverse_permute_bytes(state: bytearray):
    """
    Inverte la permutazione dei byte.
    Inverso dello shift ciclico semplice di permute_bytes.
    """
    if len(state) == 16: # Assumendo un blocco 4x4
        # Riga 1 (indici 4-7) - shift di 1 a destra
        temp_row1 = state[4:8]
        state[4:8] = temp_row1[-1:] + temp_row1[:-1]
        # Riga 2 (indici 8-11) - shift di 2 a destra
        temp_row2 = state[8:12]
        state[8:12] = temp_row2[-2:] + temp_row2[:-2]
        # Riga 3 (indici 12-15) - shift di 3 a destra
        temp_row3 = state[12:16]
        state[12:16] = temp_row3[-3:] + temp_row3[:-3]
    else:
        # Inverso della rotazione dell'intero blocco
        if len(state) > 0:
            last_byte = state.pop()
            state.insert(0, last_byte)

# --- ALGORITMO DI CIFRATURA (Blocco Singolo) ---
def encrypt_block(plaintext_block: bytes, key: bytes) -> bytes:
    """Cifra un singolo blocco di plaintext."""
    if len(plaintext_block) != BLOCK_SIZE_BYTES:
        raise ValueError(f"Il blocco di plaintext deve essere di {BLOCK_SIZE_BYTES} byte")

    round_keys = key_schedule(key)
    state = bytearray(plaintext_block)

    # Initial Round Key Addition (Whitening)
    add_round_key(state, round_keys[0])

    for i in range(1, NUM_ROUNDS + 1):
        substitute_bytes(state, S_BOX)
        permute_bytes(state)
        # MixColumns (omesso per semplicità, ma cruciale in AES per diffusione)
        add_round_key(state, round_keys[i])

    # L'ultimo round in AES omette MixColumns.
    # Qui, la nostra `permute_bytes` è l'ultima operazione prima di `add_round_key` nel loop.
    # Se si volesse mimare AES più da vicino, l'ultimo `add_round_key`
    # potrebbe essere preceduto solo da `substitute_bytes` e `permute_bytes` (ShiftRows in AES).

    return bytes(state)

# --- ALGORITMO DI DECIFRATURA (Blocco Singolo) ---
def decrypt_block(ciphertext_block: bytes, key: bytes) -> bytes:
    """Decifra un singolo blocco di ciphertext."""
    if len(ciphertext_block) != BLOCK_SIZE_BYTES:
        raise ValueError(f"Il blocco di ciphertext deve essere di {BLOCK_SIZE_BYTES} byte")

    round_keys = key_schedule(key) # Genera le stesse chiavi di round dell'encrypt
    state = bytearray(ciphertext_block)

    # I round di decifratura invertono le operazioni dei round di cifratura,
    # partendo dall'ultimo round di cifratura fino al primo.

    # Loop per i round da NUM_ROUNDS (ultimo) giù fino a 1
    for r_idx in range(NUM_ROUNDS, 0, -1):  # r_idx va da NUM_ROUNDS, NUM_ROUNDS-1, ..., 1
        # 1. Inverti AddRoundKey del round r_idx della cifratura
        add_round_key(state, round_keys[r_idx])

        # 2. Inverti PermuteBytes del round r_idx della cifratura
        #    (NB: In AES reale, l'ultimo round di cifratura omette MixColumns.
        #     Se il nostro permute_bytes fosse omesso nell'ultimo round di encrypt,
        #     allora inverse_permute_bytes dovrebbe essere saltato qui quando r_idx == NUM_ROUNDS.
        #     Ma il nostro encrypt_block attuale applica permute_bytes in tutti i round da 1 a NUM_ROUNDS)
        inverse_permute_bytes(state)

        # 3. Inverti SubstituteBytes del round r_idx della cifratura
        inverse_substitute_bytes(state, INV_S_BOX)

    # Alla fine, inverti l'AddRoundKey iniziale (whitening) che usava round_keys[0]
    add_round_key(state, round_keys[0])

    return bytes(state)


# --- MAIN (ESEMPIO DI UTILIZZO) ---
# Ricorda: questo è un esempio molto semplificato e NON SICURO per uso reale!
# Necessita di modalità operative (CBC, CTR, GCM) per messaggi più lunghi.
if __name__ == "__main__":
    # Chiave e testo in chiaro (devono essere della dimensione corretta in byte)
    my_key = b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f'
    plaintext = b'Questo e un test' # 16 byte

    if len(plaintext) != BLOCK_SIZE_BYTES:
        # Semplice padding PKCS#7 se necessario (non robusto per questo esempio)
        padding_len = BLOCK_SIZE_BYTES - (len(plaintext) % BLOCK_SIZE_BYTES)
        plaintext += bytes([padding_len] * padding_len)
        print(f"Plaintext dopo padding: {plaintext.hex()} (lunghezza: {len(plaintext)})")


    print(f"Plaintext originale: {plaintext!r}")
    print(f"Plaintext (hex): {plaintext.hex()}")

    ciphertext = encrypt_block(plaintext, my_key)
    print(f"Ciphertext (hex): {ciphertext.hex()}")

    decrypted_text = decrypt_block(ciphertext, my_key)
    print(f"Decrypted text (hex): {decrypted_text.hex()}")
    print(f"Decrypted text: {decrypted_text!r}")

    # Rimuovi il padding (se applicato)
    # padding_val = decrypted_text[-1]
    # if padding_val <= BLOCK_SIZE_BYTES and all(p == padding_val for p in decrypted_text[-padding_val:]):
    #     decrypted_text_unpadded = decrypted_text[:-padding_val]
    # else:
    #     decrypted_text_unpadded = decrypted_text # Nessun padding valido trovato

    if decrypted_text == plaintext:
        print("\nSuccesso: Plaintext e Decrypted text coincidono!")
    else:
        print("\nErrore: Plaintext e Decrypted text NON coincidono.")
        print(f"Plaintext:    {plaintext.hex()}")
        print(f"Decrypted: {decrypted_text.hex()}")

    # Esempio di come un piccolo cambiamento nel plaintext o nella chiave
    # dovrebbe cambiare drasticamente il ciphertext (effetto valanga).
    plaintext_mod = bytearray(plaintext)
    plaintext_mod[0] ^= 0x01 # Cambia un bit del primo byte
    ciphertext_mod_pt = encrypt_block(bytes(plaintext_mod), my_key)
    print(f"\nCiphertext con 1 bit di plaintext cambiato: {ciphertext_mod_pt.hex()}")

    key_mod = bytearray(my_key)
    key_mod[0] ^= 0x01 # Cambia un bit della chiave
    ciphertext_mod_key = encrypt_block(plaintext, bytes(key_mod))
    print(f"Ciphertext con 1 bit della chiave cambiato: {ciphertext_mod_key.hex()}")

Plaintext originale: b'Questo e un test'
Plaintext (hex): 51756573746f206520756e2074657374
Ciphertext (hex): b6f7dceffcda2e3675fd758621a934ba
Decrypted text (hex): 51756573746f206520756e2074657374
Decrypted text: b'Questo e un test'

Successo: Plaintext e Decrypted text coincidono!

Ciphertext con 1 bit di plaintext cambiato: dff7dceffcda2e3675fd758621a934ba
Ciphertext con 1 bit della chiave cambiato: dff7dceffcda3c36e82a945125a928ba
